In [1]:
import os
import django

PROJECT_TITLE = "app"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", PROJECT_TITLE + ".settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "True"
django.setup()

In [10]:
from django.http import QueryDict
from chat.consumers import User
from django.db.models import *

from chat.models import Chat

current_user = 1

async for i in (
    User.objects.exclude(id=current_user)
    .prefetch_related(
        Prefetch("chats", Chat.objects.filter(participants__in=[current_user]))
    ).select_related('chat_profile')
    .annotate(
        last_message=F("chats__last_message__message"),
        is_online=F("chat_profile__is_online"),
    )
    .all()
    .values("id", "last_message", "is_online")
):
    print(i)

{'id': 2, 'last_message': None, 'is_online': False}
